In [2]:
#imports and stuff
import numpy as np
import sympy as sp


In [45]:
def round_expr(expr, num_digits):
    return expr.xreplace({n : round(n, num_digits) for n in expr.atoms(sp.Number)})

def generateQUBO(N,Nt,Nq,H):
    coefficients = H.coeffs()
    monomials = [sp.prod(x**k for x, k in zip(H.gens, mon)) for mon in H.monoms()]

    Q = np.zeros((N*Nt*Nq,N*Nt*Nq))
    diag_labels = np.array([])
    for n in range(N):
        for t in range(Nt):
            for q in range(Nq):
                mlabel = 'x_'+str(n)+'_'+str(t)+'_'+str(q)
                diag_labels = np.append(diag_labels,mlabel)
    
    print(diag_labels)
    for i in range(len(monomials)):
        mon = sp.Poly(monomials[i])
        gens = np.asarray(mon.gens)
        qubo_c = coefficients[i]
        
        qubo_i = np.where(diag_labels==str(gens[0]))[0][0]
        qubo_j = np.where(diag_labels==str(gens[0]))[0][0]
        
        if (len(gens) > 1):
            qubo_j = np.where(diag_labels==str(gens[1]))[0][0]
        
        Q[qubo_i,qubo_j] += qubo_c
                
    return Q

def generateWt(t,N,Nq,K):
    w_t = np.array([])
    for n in range(N):
        q_expr = 0
        for q in range(Nq):
            q_expr = q_expr + (2**q)*sp.symbols('x_'+str(n)+'_'+str(t)+'_'+str(q))
        
        w_n_t = (1/K)*q_expr
        w_t = np.append(w_t,w_n_t)
    
    return sp.Matrix([w_t])

N = 3
Nt = 2
Nq = 1
K = 2
Ntot = N*Nt*Nq

f_returns = np.random.rand(Nt,N)
cov = np.array([np.random.rand(N,N),np.random.rand(N,N)])
risk = 1/2
l = np.random.rand()

H = 0
for t in range(Nt):
    w_t = generateWt(t,N,Nq,K)
    mu_t = -1*sp.Matrix([f_returns[t]])
    cov_t = sp.Matrix(cov[t])
    
    H = H + mu_t.dot(w_t)
    H = H + risk*(w_t*cov_t).dot(w_t)

sp.init_printing()

H = sp.Poly(sp.expand(H))
qubo = generateQUBO(N,Nt,Nq,H)

print(H)
print('----------')
print(qubo)

#tx_expr = 0
#for t in range(Nt-1):
#    tx_expr = tx_expr + l*(generateWt(t+1,N,Nq,K) - generateWt(t,N,Nq,K))*(generateWt(t+1,N,Nq,K) - generateWt(t,N,Nq,K))

#H = H + tx_expr

#QUBO = np.zeros((Ntot,Ntot))

#asset_data = np.random.rand(365,N)
#forcasted_return = np.random.rand(Nt,N)
#covariances = np.array([np.random.rand(N,N),np.random.rand(N,N)])
#tx_costs = np.array([np.random.rand(N,N),np.random.rand(N,N)])

#print(forcasted_return)

['x_0_0_0' 'x_0_1_0' 'x_1_0_0' 'x_1_1_0' 'x_2_0_0' 'x_2_1_0']
Poly(0.111304766283623*x_0_0_0**2 + 0.0648680016519118*x_0_0_0*x_1_0_0 + 0.202939386954759*x_0_0_0*x_2_0_0 - 0.451935694693166*x_0_0_0 + 0.0295186071490284*x_0_1_0**2 + 0.0822318074795079*x_0_1_0*x_1_1_0 + 0.0446904676940358*x_0_1_0*x_2_1_0 - 0.0855841759531638*x_0_1_0 + 0.0501083200203005*x_1_0_0**2 + 0.106972030229808*x_1_0_0*x_2_0_0 - 0.256204600192618*x_1_0_0 + 0.0274209073039754*x_1_1_0**2 + 0.115377401295341*x_1_1_0*x_2_1_0 - 0.377298957125988*x_1_1_0 + 0.124687486140489*x_2_0_0**2 - 0.46723474245774*x_2_0_0 + 0.108192864114045*x_2_1_0**2 - 0.110824041458911*x_2_1_0, x_0_0_0, x_0_1_0, x_1_0_0, x_1_1_0, x_2_0_0, x_2_1_0, domain='RR')
----------
[[-0.34063093  0.          0.064868    0.          0.20293939  0.        ]
 [ 0.         -0.05606557  0.          0.08223181  0.          0.04469047]
 [ 0.          0.         -0.20609628  0.          0.10697203  0.        ]
 [ 0.          0.          0.         -0.34987805  0.  

In [46]:
0.108192864114045-0.110824041458911

In [15]:
def Q1(forcasted_return):
    Q1_QUBO = np.zeros((Ntot,Ntot))
    np.fill_diagonal(Q1_QUBO,-1*forcasted_return)
        
    return Q1_QUBO

print(Q1(forcasted_return))

NameError: name 'forcasted_return' is not defined